# Logger Wrapper personalization/configuration

We will see on this notebook how to personalize the logger wrapper defined by sinergym.

In [9]:
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.constants import RANGES_5ZONE

## Step 1 Inherit and modify the CSVloger

First we need to change the CSV logger to modify the values written into the file on the function create_row_contents

In [10]:

from sinergym.utils.logger import CSVLogger
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

class CustomCSVLogger(CSVLogger):

    def __init__(
            self,
            monitor_header: str,
            progress_header: str,
            log_progress_file: str,
            log_file: Optional[str] = None,
            flag: bool = True):
        super(CustomCSVLogger, self).__init__(monitor_header,progress_header,log_progress_file,log_file,flag)
        self.last_10_steps_reward = [0]*10

    def _create_row_content(
            self,
            obs: List[Any],
            action: Union[int, np.ndarray, List[Any]],
            terminated: bool,
            info: Optional[Dict[str, Any]]) -> List:
            
        if info.get('reward') is not None:
            self.last_10_steps_reward.pop(0)
            self.last_10_steps_reward.append(info['reward'])

        
        return [
            info.get('timestep',0)] + list(obs) + list(action) + [
            info.get('time_elapsed(hours)',0),
            info.get('reward',None),
            np.mean(self.last_10_steps_reward),
            info.get('total_power_no_units'),
            info.get('comfort_penalty'),
            info.get('abs_comfort'),
            terminated]

    

## Step 2 Instantiate the LoggerWrapper


now we need to instantiate the loggerwrapper and specify the new headers of our file and the csvlogger class we want to use.

In [11]:
env=gym.make('Eplus-demo-v1')
env=LoggerWrapper(env,logger_class=CustomCSVLogger,monitor_header = ['timestep'] + env.observation_variables +
                env.action_variables + ['time (hours)', 'reward', '10-mean-reward',
                'power_penalty', 'comfort_penalty', 'done'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [demo-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-demo-v1-res5]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment demo-v1 created successfully.
[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.


Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

In [12]:
for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-demo-v1-res5/Eplus-env-sub_run1]
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-demo-v1-res5/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-demo-v1-res5/Eplus-env-sub_run1/USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-demo-v1-res5/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-demo-v1-res5/Eplus-env-sub_run1/5ZoneAutoDXVAV.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER LoggerWrapper] (INFO) : Creating monitor.csv for current episode (episode 1) if logger is active
Reward:  -1.4254385096063296 {'time_elapsed(hours)': 0.75, 'year': 1995, 'month': 1, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [22.455776, 26.17595], 'timestep': 2, 'reward': -1.4254385096063296, 'reward_energy': -0.09759966457086716, 'reward_comfort': -2.753277354641792, 'total_energy': 975.9966457086716, 'abs_comfort': 2.753277354641792, 'temperatures': [17.246722645358208]}
Reward:  -1433.4400729417541 {'time_elapsed(hours)': 744.3, 'year': 2004, 'month': 2, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [17.686495, 24.52391], 'timestep': 2976, 'reward': -0.6539268966156082, 'reward_energy': -0.27759966457086716, 'reward_comfort': -1.0302541286603493, 'total_energy': 2775.9966457086716, 'abs_comfort': 1.0302541286603493, 'temperatures': [18.96974